In [37]:
#!usr/bin/env python

from __future__ import print_function

In [38]:
from pyrosetta import *
from rosetta import *

# tools
import sys, os
import commands
import random
import numpy as np

# rosetta corrected imports
import rosetta.protocols.membrane
from rosetta.core.pose import Pose
from rosetta.core.scoring import ScoreFunction
from rosetta.core.pack.task import TaskFactory
from rosetta.utility import vector1_bool
from rosetta.core.chemical import aa_from_one_or_three
from rosetta.protocols.minimization_packing import PackRotamersMover
from rosetta.core.pose import PDBInfo
from rosetta.core.chemical import VariantType
from pyrosetta.toolbox import mutate_residue
from pyrosetta.rosetta.core import conformation
from pyrosetta.rosetta.core import chemical

In [23]:
ddG?

In [39]:
nAAs = ['ALA','ASP','LEU','ILE','VAL','GLY','SER','THR','PRO','GLU','ASN','GLN','LYS','ARG','TRP','PHE','TYR','HIS','CYS','MET']
nsAAs = ['R1A']

In [3]:
class AbstractExperimentRunner():

    def __init__(self,start_pose_pdbs,rosetta_init_options,comm,restrict_to_chain,max_pack_rounds,min_cst_sd,min_restrict_radius,PDB_res,dump_ref_pdb,dump_mut_pdb,pdb_base,verbose,constraint_file):

        if verbose >= 2:
            print >> sys.stderr, "AbstractExperimentRunner __init__(): restrict_to_chain= %s" % (str(restrict_to_chain),)

        self.start_pose_pdbs = start_pose_pdbs
        self.rosetta_init_options = rosetta_init_options
        self.max_pack_rounds = max_pack_rounds
        self.restrict_to_chain = restrict_to_chain
        self.pdb_res = PDB_res
        self.min_cst_sd = min_cst_sd
        self.min_restrict_radius = min_restrict_radius
        self.dump_ref_pdb = dump_ref_pdb
        self.dump_mut_pdb = dump_mut_pdb
        self.pdb_base = pdb_base
        self.verbose = verbose
        self.constraint_file = constraint_file
        self.comm = None
        self.size = 1
        self.rank = 0

#     def setup_MPI(self,comm):
#         if not comm:
#             pyrosetta.init(extra_options=self.rosetta_init_options)
#         else:
#             self.comm = comm
#             self.size = comm.Get_size()
#             self.rank = comm.Get_rank()
#         # keep a list of Job objects
#             mpi_init(extra_options=self.rosetta_init_options)
            
#     def scatter_job(self):
#         local_jobs = self.comm.scatter(self.jobs,root=0)
#         for (i,job_spec) in enumerate(local_jobs):
#             if self.verbose >=1:
#                 print "===== Process %d, running job %s [ %d / %d ]" % (self.rank,str(job_spec),i,len(local_jobs))
#             try:
#                 ddg_job = self.packer_job_class(*job_spec,max_rounds=self.max_pack_rounds,restrict_to_chain=self.restrict_to_chain,min_restrict_radius=self.min_restrict_radius,PDB_res=self.pdb_res,min_cst_sd=self.min_cst_sd,verbose=self.verbose,constraint_file=self.constraint_file)
            
#                 ddg_job.run()
#                 self.result.append(ddg_job.get_result())
#                 if self.dump_ref_pdb:
#                     ddg_job.dump_ref_pdb(self.pdb_base + "_".join([ddg_job.start_pose_name] + [str(x) for x in job_spec[1:]]) + "_ref.pdb")
#                 if self.dump_mut_pdb:
#                     start_pose_idx = "p" + str(self.start_pose_pdbs.index(job_spec[0]))
#                     ddg_job.dump_mut_pdb(self.pdb_base + "_".join([ddg_job.start_pose_name] + [str(x) for x in job_spec[1:]]) + "_mut.pdb")
#             except RuntimeError as e:
#                 print >> sys.stderr, "****WARNING: RUNTIME ERROR IN PROCESS %d JOB %s; ABORTING RUN [ERROR:%s]" % (self.rank,str(job_spec),e)
#                 sys.exit(0)
#             except PyRosettaError as e:
#                 print >> sys.stderr, "****WARNING: PYROSETTA ERROR IN PROCESS %d JOB %s; SKIPPING JOB [ERROR:%s]" % (self.rank,str(job_spec),e)

#     def gather_results(self):
#         results = self.comm.gather(self.result,root=0)
#         if self.rank == 0:
#             for r in results:
#                 self.final_results.extend(r)
    
#     def run_all_jobs_serial(self,outfile='ddg_out.txt'):
#         for (i,job_spec) in enumerate(self.jobs):
#             if self.verbose >= 1:
#                 print "===== Process %d, running job %s [ %d / %d ]" % (self.rank,str(job_spec),i,len(local_jobs))
#             ddg_job = MutantddGPackerJob(*job_spec,max_rounds=self.max_pack_rounds,restrict_to_chain=self.restrict_to_chain,PDB_res=self.pdb_res)
#             try:
#                 ddg_job.run()
#             except RuntimeError:
#                 print >> sys.stderr, "****WARNING: RUNTIME ERROR IN PROCESS %d JOB: %s %s; SKIPPING JOB" % (self.rank,str(job_spec),ddg_job.repr_str)
#             self.final_results.append(ddg_job.get_result())
#         self.dump_ddg_results(outfile)
    
#     def dump_ddg_results(self,outfile_name,header=True):
#         """
#         Specification Required!
#         """
#         pass

In [34]:
class AbstractPackerJob():
    
    def __init__(self,convergence_fn=None,\
                 conv_threshold=0.1,repack_radius=10,scorefn="mm_std",\
                 mintype="dfpmin_armijo_nonmonotone",max_rounds=100,restrict_to_chain=None,verbose=1,constraint_file=None):
        self.convergence_fn = convergence_fn
        self.cnv_fn = None
        self.conv_threshold = conv_threshold
        if not convergence_fn or convergence_fn == 'stdev': # defaults to stdev for now, may add more later
            self.cnv_fn = self.std_dev_threshold_fn_builder(conv_threshold)
        self.max_rounds = max_rounds    
        self.repack_radius = repack_radius
        self.mintype = mintype
        self.restrict_to_chain = restrict_to_chain
        self.verbose = verbose
        # a PyRosetta ScoreFunction object, defaults to mm_std    
        self.scorefn = pyrosetta.create_score_function(scorefn)        
        self.constraint_file = constraint_file

    def mutate_aa(self,pose,residue,aa_name,orient_bb=True,repack_sidechain=True,clone_pose=True):
        """
        Swap w/t AA at residue number 'residue' in 'pose' with 'ncaa_name' (3-letter code)
    
        Return a new Pose object
        
        Note that this assumes the ncaa .params and .rotlib files have been permanently added to the database
        """

        mut_pose = pose
        if clone_pose:
            mut_pose = pyrosetta.Pose()
            mut_pose.assign(pose)

        res = mut_pose.residue(residue)
        ref_res_name = res.name()
        # check for disulfides and correct if needed
        if (res.name() == 'CYS:disulfide') or (res.name() == 'CYD'):
            disulfide_partner = None
            try:
                disulfide_partner = res.residue_connection_partner(
                    res.n_residue_connections())
            except AttributeError:
                disulfide_partner = res.residue_connection_partner(
                    res.n_current_residue_connections())
            temp_pose = pyrosetta.Pose()
            temp_pose.assign(mut_pose)
            # (Packing causes seg fault if current CYS residue is not
            # also converted before mutating.)
            conformation.change_cys_state(residue, 'CYS',\
                             temp_pose.conformation())
            conformation.change_cys_state(disulfide_partner, 'CYS',\
                             temp_pose.conformation())
            mut_pose = temp_pose

    
            # Get a Residue object for the desired ncAA
        mut_res_type = None
        rts = None
        chm = chemical.ChemicalManager.get_instance()
        
        try:
            rts = chm.residue_type_set("fa_standard").get()
        except AttributeError:
            # older versions of PyRosetta (e.g. the TACC stampede install of 3.4) 
            # have a slightly different call to get the residue_type_set object:
            rts = chm.residue_type_set("fa_standard")
    
            
#         if aa_name in NSAAS_PATCH.keys():
#             # PTMs and nsAAs using the patch system need to be treated differently
#             # for TACC PyRosett install; don't know if this will work with newer versions
#             # where VariantType enumerator class can be called direct
#             cognate_res_type = rts.name_map( NSAAS_PATCH[aa_name]['cognateAA'] ) 
#             mut_res_type = rts.get_residue_type_with_variant_added(cognate_res_type,NSAAS_PATCH[aa_name]['type'])            
#         else:
#             # replace the target residue with the ncAA
#             #if residue == 1 or residue == mut_pose.total_residue():
#             #    aa_name = aa_name + "_p"
#             mut_res_type = rts.name_map(aa_name)

        if residue == 1:
            try: 
                mut_res_type = rts.get_residue_type_with_variant_added(mut_res_type,chemical.VariantType.LOWER_TERMINUS_VARIANT)                
            except AttributeError:
                mut_res_type = rts.get_residue_type_with_variant_added(mut_res_type,"LOWER_TERMINUS")
        elif residue == mut_pose.total_residue():
            try:
                mut_res_type = rts.get_residue_type_with_variant_added(mut_res_type,chemical.VariantType.UPPER_TERMINUS_VARIANT)                
            except AttributeError:
                mut_res_type = rts.get_residue_type_with_variant_added(mut_res_type,"UPPER_TERMINUS")
        mut_res = conformation.ResidueFactory.create_residue( mut_res_type )
        mut_pose.replace_residue(residue,mut_res,orient_backbone=orient_bb)

        # Highly recommended to repack the sidechain after calling Pose.replace_residue()...otherwise
        # later packing steps get caught in weird local minima :'(
        #tmp_verbose = self.verbose
        #self.verbose=2
        if repack_sidechain:
            # annoying but apparently has to be done this way?...
            repack_task = pyrosetta.rosetta.core.pack.task.TaskFactory.create_packer_task(mut_pose)
            repack_task.restrict_to_repacking()
            repack_res_list = pyrosetta.rosetta.utility.vector1_bool()
            for i in range(1,mut_pose.total_residue()+1):
                repack_res_list.append(i == residue)
            repack_task.restrict_to_residues(repack_res_list)
            sidechain_PackRotamersMover = pyrosetta.rosetta.protocols.simple_moves.PackRotamersMover(self.scorefn,repack_task)
            
            initial_mut_score = self.scorefn(mut_pose)
            if self.verbose > 1:
#                 print "Packer task"
                repack_task
                #print "Repacking mutated sidechain %s %s -> %s..." % 
                print (mut_pose.pdb_info().pose2pdb(residue),ref_res_name,aa_name),
            sidechain_PackRotamersMover.apply(mut_pose)
            repacked_mut_score = self.scorefn(mut_pose)
            if self.verbose > 1:
#                print "initial E = %f repacked E = %f" % 
                print (initial_mut_score,repacked_mut_score)
        #self.verbose=tmp_verbose

        return mut_pose

        
    def packer_task_repack_in_radius(self,pose,residue,radius):
        """
        Build a packer task that repacks all residues with CAlphas within distance radius
    
        Might want to remake to take centroid side-chains?  Looks like SwitchResidueTypeMover has trouble w/ nsAAs...
        """
        pack_residues = self.residue_CAs_in_radius(pose,residue,radius)
        return self.make_packer_task_with_residues(pose,pack_residues)

    def make_packer_task_with_residues(self,pose,residues=None):
        """
        Builds a packer task with the specified residues activated for repacking.
    
        Did this to avoid PackerTask.temporarily_* methods which apparently we're not supposed to use
        """
        packer_task = pyrosetta.rosetta.core.pack.task.TaskFactory.create_packer_task(pose)
        packer_task.restrict_to_repacking()
        if self.verbose > 1:
#             print residues
            residues
        if residues != None:
            # Vector1 doesn't translate booleans correctly in 
            # TACC version of PyRosetta; need to build utility.vector1_bool() directly
            #pack_residues = Vector1([x in residues for x in range(1,pose.total_residue())])
            pack_residues = pyrosetta.rosetta.utility.vector1_bool()
            for i in range(1,pose.total_residue() + 1):
                if self.restrict_to_chain and not (pose.pdb_info().chain(i) in self.restrict_to_chain):
                    continue
                else:
                    #print (i,i in residues)
                    pack_residues.append(i in residues)
            packer_task.restrict_to_residues(pack_residues)
        
        return packer_task

    def residue_CAs_in_radius(self,pose,centerAA,radius):
        """
        Get a list of residues with C-alpha atoms within 'radius' distance from centerAA'a C-alpha
        """
        centerAA_CA = pose.residue(centerAA).xyz('CA')
        repack_residues = []
        if self.verbose > 2:
            print >> sys.stderr, "residue_CA_in_radius: centerAA=%d, restrict_to_chain=%s" % (centerAA,str(self.restrict_to_chain))
        for i in range(1,pose.total_residue()):
            # This is a little fudgy since restrict_to_chain could be either a list or a string depending on the calling child class
            # 
            # Note that this may break if chains are not single characters
            if self.restrict_to_chain and not (pose.pdb_info().chain(i) in self.restrict_to_chain):
                continue
            test_CA = pose.residue(i).xyz('CA')
            displacement = centerAA_CA - test_CA
            distance = displacement.norm()
            if distance <= radius:
                repack_residues.append(i)
    
        return repack_residues
    
    def make_CompoundMover(self,movers,repeats_per_round):
        
        class CompoundMover:

            def __init__(self,movers,repeats):
                self.movers = movers
                self.repeats = repeats

            def apply(self,pose):
                for (mv,reps) in zip(self.movers,self.repeats):
                    for i in range(0,reps):
                        mv.apply(pose)
        
        return CompoundMover(movers,repeats_per_round)

    def make_packmin_mover(self,pose,packertask,minmover,n_packing_steps,n_minimize_steps,kT=1.0):
        """
        Build a TrialMover with n_packing_steps RotamerTrialMover moves and 
            n_minimization_steps MinMover moves executed sequentially
        """
        
        seqmover = pyrosetta.rosetta.protocols.moves.SequenceMover()
        packmover = pyrosetta.rosetta.protocols.simple_moves.PackRotamersMover(self.scorefn,packertask)

        for i in range(0,n_packing_steps):
            #pack_repmover = rosetta.RepeatMover(packmover,n_packing_steps)
            seqmover.add_mover(packmover)
        
        for i in range(0,n_minimize_steps):
            #min_repmover = rosetta.RepeatMover(minmover,n_minimize_steps)
            seqmover.add_mover(minmover)
         

        #print >> sys.stderr, seqmover
        
        mc = pyrosetta.rosetta.protocols.moves.MonteCarlo(pose,self.scorefn,kT)
        packmin_trial_mover = pyrosetta.rosetta.protocols.moves.TrialMover(seqmover,mc)
    
        return seqmover
    
    def std_dev_threshold_fn_builder(self,threshold,last_n=5):
        def stop(scores):
            passed = False
            if len(scores) > last_n:
                last_n_stdev = np.std(scores[-last_n:])
                passed = last_n_stdev < threshold
                if self.verbose >= 2:
#                     print "std dev for scores %s: %f thresh: %f pass?: %s" % 
                    print (str(scores[-last_n:]),last_n_stdev,  threshold,str(passed))
            return passed
        return stop

    def build_movemap(self,pose,restrict_radius_center=None,restrict_radius=None,restrict_residues=None):
        if not restrict_radius:
            restrict_radius = self.repack_radius
        mm = pyrosetta.rosetta.core.kinematics.MoveMap()
        mm.set_bb(False)
        mm.set_chi(False)
        mm.set_jump(False)
        residues = []
        if self.restrict_to_chain:
            for i in range(1,pose.total_residue()):
                if pose.pdb_info().chain(i) in self.restrict_to_chain:
                    residues.append(i)
        else:
            residues = range(1,pose.total_residue() + 1)
        
        if restrict_radius_center:
            possible_residues = self.residue_CAs_in_radius(pose,restrict_radius_center,restrict_radius)
            restricted_residues = [r for r in possible_residues if r in resiudes]
            residues = restricted_residues
        elif restrict_residues:
            restricted_residues = [r for r in restrict_residues if r in residues]
            residues = restricted_residues

        for r in residues:
            mm.set_bb(r,True)
            mm.set_chi(r,True)
        
        return mm
        
    def make_minmover(self,mintype,movemap,pose):
        # Set up a Minimization mover using the mm_std score function
        min_mover = pyrosetta.rosetta.protocols.simple_moves.MinMover()
        min_mover.movemap(movemap)
        min_scorefn = self.scorefn.clone()
        if self.min_cst_sd:
            min_scorefn.set_weight(pyrosetta.rosetta.core.scoring.coordinate_constraint,1.0) # arbitrary weight for now
        min_mover.score_function(min_scorefn)
        min_mover.min_type(mintype)
        
        return min_mover

    def add_constraints_to_scorefxn(self,constraint_types=None,weights=None,default_weight=0.1):
        if not constraint_types:
            constraint_types = [pyrosetta.rosetta.core.scoring.constraints.atom_pair_constraint, \
                                pyrosetta.rosetta.core.scoring.constraints.angle_constraint, \
                                pyrosetta.rosetta.core.scoring.constraints.dihedral_constraint, \
                                pyrosetta.rosetta.core.scoring.constraints.coordinate_constraint, \
                                pyrosetta.rosetta.core.scoring.constraints.constant_constraint]
        if not weights:
            weights = [default_weight,] * len(constraint_types)

        for (constraint,weight) in zip(constraint_types,weights):
            if self.scorefn.get_weight(constraint)==0:
                self.scorefn.set_weight(constraint, weight)
    
    def add_constraints_to_pose(self,pose,constraint_file):
        # Still not sure how to implement this in PyRosetta4 ?
        #setup = rosetta.ConstraintSetMover()
        #setup.constraint_file(constraint_file)
        #setup.apply(pose)
        pyrosetta.rosetta.core.scoreing.constraints.add_constraints_from_cmdline_to_pose(pose)

In [35]:
init()

Found rosetta database at: /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta PyRosetta4.Release.python27.ubuntu 2018.12+release.5ecebca5ecebcadacdf48ede3c1981444601bf1cd47ce0d 2018-03-23T13:02:49] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Checking for fconfig files in pwd and ./rosetta/flags

core.init: Rosetta version: PyRosetta4.Release.python27.ubuntu r174 2018.12+release.5ecebca 5ecebcadacdf48ede3c1981444601bf1cd47ce0d http://www.pyrosetta.org 2018-03-23T13:02:49
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.12+release.5ecebca-py2.7-linux-x86_64.egg/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=1801339003 seed_offset=0 real_seed=1801339003
core.init.random: RandomGenerator

In [42]:
#mutate_aa(self,pose,residue,aa_name,orient_bb=True,repack_sidechain=True,clone_pose=True):
"""
Swap w/t AA at residue number 'residue' in 'pose' with 'ncaa_name' (3-letter code)
    
Return a new Pose object
        
Note that this assumes the ncaa .params and .rotlib files have been permanently added to the database
"""
pose = pose_from_file("TCM_pR.pdb")
residue = "167"
aa_name = "R1A"


core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 606 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.890391 seconds.
core.import_pose.import_pose: File 'TCM_pR.pdb' automatically determined to be of type PDB
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type RYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type DRYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type DRYS, but actually applying it fails.
core.chemical: [ WARNING ]    You may want to check your patch definitions.
core.chemical: [ WARNING ] Patch N_Methylation implies it can apply to residue type DRYS, but actually applying it fails.


In [43]:
mutate_aa(self,pose,residue,aa_name,orient_bb=True,repack_sidechain=True,clone_pose=True

SyntaxError: unexpected EOF while parsing (<ipython-input-43-5f2ba5fb4be1>, line 1)